# Introduction

This Recommender System recommends similar hostels with an explanation and as a result, such recommendations become more effective and more persuasive.

Since there is no data available for hostels in Ireland on data libraries so I have scrapped the data from Hostel World website for the experiment.

Let's start by importing the necessary libraries.

In [ ]:
# Importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import euclidean_distances
from IPython.display import display, HTML

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Loading the dataset
df_hostels = pd.read_csv("../input/explainable_recommendations.csv", encoding='latin1')

In [ ]:
df_hostels.head()

Here for some entertainment features, the values are give as 1 and 0. It means 0 = No and 1 = Yes.

# 1. Exploratory Data Analysis

In [ ]:
# General Information
df_hostels.info()

In [ ]:
# Statistical characteristics of numerical features
df_hostels.describe()

Let's draw histograms for some relavent fields

In [ ]:
plt.figure(figsize=(14, 10))
plt.subplot(221)
plt.hist(df_hostels['Price'].values, bins=20)
plt.title('Price')
plt.subplot(222)
plt.hist(df_hostels['summary.score'].values, bins=20)
plt.title('summary.score')

Most hostels in the dataset are between 15 to 20 Euros.

# 2. Data Preprocessing

Handling Missing Values

In [ ]:
df_hostels.isnull().sum()

No missing value are found.

Now, I'm deleting summary.score, Name, and rating.band. Because summary.score is the average value for Value.for.money, Security, Location, Staff, Atmosphere, Cleanliness, and Facilities. And I don't think ranting.band is necessary because they're giving rating based on summary.score i-e if summary.score is between 1 to 3, then rating.band is "Good".

In [ ]:
df_hostels.drop(['summary.score', 'Name', 'rating.band'], inplace=True, axis=1);

In [ ]:
df_hostels.head()

Now, label encoding categorical features like Distance and City.

In [ ]:
# Label Encoding
le = LabelEncoder()
df_hostels['Distance'] = le.fit_transform(df_hostels['Distance'])
df_hostels['City'] = le.fit_transform(df_hostels['City'])

df_hostels.head(3)

In [ ]:
def get_hostel_recommendations(df, anchor_id):
    # features used to compute the similarity
    features = ['City']
    
    # create the features - make the anchor be the first row in the dataframe
    df_sorted = df.copy()
    df_sorted = pd.concat([df_sorted[df_sorted['hostel.id'] == anchor_id],df_sorted[df_sorted['hostel.id'] != anchor_id]])
    df_features = df_sorted[features].copy()
    df_features = normalize_features(df_features)
    
    # compute the distances
    X = df_features.values
    Y = df_features.values[0].reshape(1, -1)
    distances = euclidean_distances(X, Y)
    
    df_sorted['similarity_distance'] = distances
    new_df = df_sorted.sort_values('similarity_distance').reset_index(drop=True)
    
    return new_df

def get_city_hostel_recommendations(df, city_id):
    features = ['Distance', 'Value.for.money', 'Security', 'Location', 'Staff', 'Atmosphere', 'Cleanliness', 'Facilities', 'Price', 'Board.Games', 'Dvds', 'Foosball', 'Games.Room', 'PlayStation', 'Pool.Table']
    
    df_sorted = df[df["City"].isin(city_id)]
    df_features = df_sorted[features].copy()
    df_features = normalize_features(df_features)
    
    # compute the distances
    X = df_features.values
    Y = df_features.values[0].reshape(1, -1)
    distances = euclidean_distances(X, Y)
    df_sorted['similarity_distance'] = distances
    return df_sorted.sort_values('similarity_distance').reset_index(drop=True)
    
def normalize_features(df):
    for col in df.columns:
        # fill any NaN's with the mean
        df[col] = df[col].fillna(df[col].mean())
        df[col] = StandardScaler().fit_transform(df[col].values.reshape(-1, 1))
    return df

def Remove(duplicate): 
    final_list = [] 
    for num in duplicate: 
        if num not in final_list: 
            final_list.append(num) 
    return final_list 

You can see I'm getting recommendations based on features = ['Distance', 'City', 'Value.for.money', 'Security', 'Location', 'Staff', 'Atmosphere', 'Cleanliness', 'Facilities', 'Price', 'Board.Games', 'Dvds', 'Foosball', 'Games.Room', 'PlayStation', 'Pool.Table'].

## Explainable Processing Methods

Now, I will code to put an explanation before the hostels' recommendations according to this logic.

If all three recommended hostels have 9 or more than 9 rating in any of the following columns (Value.for.money, Security, Location, Staff, Atmosphere, Cleanliness, Facilities), then mention those in the explanation.

Also, If all three recommended hostels have 1 which means yes in any of the following columns (Board.Games, Dvds, Foosball, Games.Room, PlayStation, Pool.Table), then mention those this in the explanation.

In [ ]:
def processSummary(df):
    tf = (df.values > 8.9)
    if False in tf[:]:
        return False
    else:
        return True
def processEntertainment(df):
    tf = (df.values == 1)
    if False in tf[:]:
        return False
    else:
        return True
def getExplaination(finalShowDown):
    # Summary Fields
    value_for_money = processSummary(finalShowDown['Value.for.money'])
    security = processSummary(finalShowDown['Security'])
    staff = processSummary(finalShowDown['Staff'])
    atmosphere = processSummary(finalShowDown['Atmosphere'])
    clean = processSummary(finalShowDown['Cleanliness'])
    facilities = processSummary(finalShowDown['Facilities'])
    location = processSummary(finalShowDown['Location'])
    
    summary = ""
    finalDecision = "Similar hostels "
    if(value_for_money):
        summary+='Value for money, '
    if(security):
        summary+='Security, '
    if(staff):
        summary+='Staff, '
    if(atmosphere):
        summary+='Atmosphere, '
    if(clean):
        summary+='Cleanliness, '
    if(facilities):
        summary+='Facilities, '
    if(location):
        summary+='Location, '

    if summary != "":
        summary = removeLastOccurence(summary, ",")
        summary_split = summary.split(",")
        if(len(summary_split) > 1):
            summary = removeAgainOccurence(summary, ",")
        finalDecision+="who are famous for excellent "+summary
        
    # Entertainment Fields
    board_games = processEntertainment(finalShowDown['Board.Games'])
    dvd = processEntertainment(finalShowDown['Dvds'])
    foosball = processEntertainment(finalShowDown['Foosball'])
    games_room = processEntertainment(finalShowDown['Games.Room'])
    play_station = processEntertainment(finalShowDown['PlayStation'])
    pool_table = processEntertainment(finalShowDown['Pool.Table'])
    
    entertainment = ""
    if(board_games):
        entertainment+='Board Games, '
    if(dvd):
        entertainment+='DVDs, '
    if(foosball):
        entertainment+='Foosball, '
    if(games_room):
        entertainment+='Games Room, '
    if(play_station):
        entertainment+='PlayStation, '
    if(pool_table):
        entertainment+='Pool Table, '

    if entertainment != "":
        entertainment = removeLastOccurence(entertainment, ",")
        ent_split = entertainment.split(",")
        if(len(ent_split) > 1):
            entertainment = removeAgainOccurence(entertainment, ",")
        if summary != "":
            finalDecision+="and also have "+entertainment+"."
        else:
            finalDecision+="who have "+entertainment+"."
    else:
        finalDecision+="."
    return removeLastOccurence(finalDecision, " ")

def removeLastOccurence(str_val, delimiter):
    k = str_val.rfind(delimiter)
    new_string = str_val[:k] + "" + str_val[k+1:]
    return new_string
def removeAgainOccurence(str_val, delimiter):
    k = str_val.rfind(delimiter)
    new_string = str_val[:k] + " and" + str_val[k+1:]
    return new_string

# Results for all hostel ids using loop

In [ ]:
for i in range(1,101):
    df_recs = get_hostel_recommendations(df_hostels, i)
    df_final = df_recs.head(n=4)
    city = Remove(df_final.City)

    # Now make recommendations for only the data where city is fetched from above
    df_final = get_city_hostel_recommendations(df_hostels, city)

    finalShowDown = df_final.head(n=3)
    print("\n")
    print('\033[1m'+"Test Resuts for hostel.id="+str(i)+"\n")
    print(getExplaination(finalShowDown))
    display(finalShowDown)